# import

## 추가 설치 사항
- matplotlib와 numpy 버젼 안맞는 오류가 나서 numpy 버젼을 올려줌
- conda install numpy==1.16
- pip install opencv-python
- pip install opencv-contrib-python
- 재부팅 후 import matplotlib.pyplot as plt 관련 ModuleNotFoundError: No module named 'cycler'에러가 발생했으나 conda install cycler 로 해결

In [1]:
import torchvision
import torch

import os
import numpy as np
from PIL import Image
from torch.utils import data
from torch.utils.data import DataLoader #데이터 불러오기
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
#훈련
from torch import nn
from torch.nn import functional as F
#시각화



In [2]:
import socket
import timeit
from datetime import datetime
import os
import glob
from collections import OrderedDict
import numpy as np
import yaml
from addict import Dict
import argparse

In [3]:
# PyTorch includes
import torch
from torch.autograd import Variable
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

In [4]:
# Tensorboard include
from tensorboardX import SummaryWriter

In [5]:
'''
# Custom includes
from dataloaders import cityscapes
from dataloaders import utils
from dataloaders import augmentation as augment
from models.liteseg import LiteSeg
from utils import loss as losses
from utils import iou_eval
'''

'\n# Custom includes\nfrom dataloaders import cityscapes\nfrom dataloaders import utils\nfrom dataloaders import augmentation as augment\nfrom models.liteseg import LiteSeg\nfrom utils import loss as losses\nfrom utils import iou_eval\n'

In [6]:
print(torch.__version__)

0.4.1


### LiteSeg-master/dataloaders/utils.py

In [7]:
#LiteSeg-master/dataloaders/utils.py
import os

import torch
import numpy as np
import matplotlib.pyplot as plt

def listFiles(rootdir='.', suffix='png'):
    """Performs recursive glob with given suffix and rootdir
        :param rootdir is the root directory
        :param suffix is the suffix to be searched as PNG or JPG
    """
    return [os.path.join(looproot, filename)
        for looproot, _, filenames in os.walk(rootdir)
        for filename in filenames if filename.endswith(suffix)]

cityscapes_valid_classes = [7, 8, 11, 12, 13, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33]
class_map = dict(zip( range(19),cityscapes_valid_classes))

def convertTrainIdToClassId(img):
         temp=np.copy(img)
         for trainID in range(19):
             #print(trainID," " ,class_map[trainID])
             temp[img==trainID]=class_map[trainID]
             
         return temp


def get_cityscapes_labels():
    return np.array([
         #[  0,   0,   0],
        [128, 64, 128],
        [244, 35, 232],
        [70, 70, 70],
        [102, 102, 156],
        [190, 153, 153],
        [153, 153, 153],
        [250, 170, 30],
        [220, 220, 0],
        [107, 142, 35],
        [152, 251, 152],
        [0, 130, 180],
        [220, 20, 60],
        [255, 0, 0],
        [0, 0, 142],
        [0, 0, 70],
        [0, 60, 100],
        [0, 80, 100],
        [0, 0, 230],
        [119, 11, 32]
        ])



def encode_segmap(mask):
    """Encode segmentation label images as pascal classes
    Args:
        mask (np.ndarray): raw segmentation label image of dimension
          (M, N, 3), in which the Pascal classes are encoded as colours.
    Returns:
        (np.ndarray): class map with dimensions (M,N), where the value at
        a given location is the integer denoting the class index.
    """
    mask = mask.astype(int)
    label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int16)
    for ii, label in enumerate(get_cityscapes_labels()):#get_pascal_labels()
        label_mask[np.where(np.all(mask == label, axis=-1))[:2]] = ii
    label_mask = label_mask.astype(int)
    return label_mask


def decode_seg_map_sequence(label_masks, dataset='pascal'):
    rgb_masks = []
    for label_mask in label_masks:
        rgb_mask = decode_segmap(label_mask, dataset)
        rgb_masks.append(rgb_mask)
    rgb_masks = torch.from_numpy(np.array(rgb_masks).transpose([0, 3, 1, 2]))
    return rgb_masks

def decode_segmap(label_mask, dataset, plot=False):
    """Decode segmentation class labels into a color image
    Args:
        label_mask (np.ndarray): an (M,N) array of integer values denoting
          the class label at each spatial location.
        plot (bool, optional): whether to show the resulting color image
          in a figure.
    Returns:
        (np.ndarray, optional): the resulting decoded color image.
    """
    if dataset == 'pascal':
      print()
    elif dataset == 'cityscapes':
        n_classes = 19
        label_colours = get_cityscapes_labels()
    else:
        raise NotImplementedError

    r = label_mask.copy()
    g = label_mask.copy()
    b = label_mask.copy()
    for ll in range(0, n_classes):
        r[label_mask == ll] = label_colours[ll, 0]
        g[label_mask == ll] = label_colours[ll, 1]
        b[label_mask == ll] = label_colours[ll, 2]
    
    r[label_mask == 255] = 0
    g[label_mask == 255] = 0
    b[label_mask == 255] =0
    
    rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
   # rgb = np.zeros((label_mask.shape[1], label_mask.shape[2], 3))
    #replace blue with red as opencv uses bgr
    rgb[:, :, 0] = r /255.0     
    rgb[:, :, 1] = g /255.0
    rgb[:, :, 2] = b /255.0
#    
#    rgb = np.zeros((label_mask.shape[1], label_mask.shape[2], 3))
#    #replace blue with red as opencv uses bgr
#    rgb[:, :, 0] = b #/255.0     
#    rgb[:, :, 1] = g #/255.0
#    rgb[:, :, 2] = r #/255.0
#    
    if plot:
        plt.imshow(rgb)
        plt.show()
    else:
        return rgb
def decode_segmap_cv(label_mask, dataset, plot=False):
    """Decode segmentation class labels into a color image
    Args:
        label_mask (np.ndarray): an (M,N) array of integer values denoting
          the class label at each spatial location.
        plot (bool, optional): whether to show the resulting color image
          in a figure.
    Returns:
        (np.ndarray, optional): the resulting decoded color image.
    """
    if dataset == 'pascal':
      print()
    elif dataset == 'cityscapes':
        n_classes = 19
        label_colours = get_cityscapes_labels()
    else:
        raise NotImplementedError

    r = label_mask.copy()
    g = label_mask.copy()
    b = label_mask.copy()
    for ll in range(0, n_classes):
        r[label_mask == ll] = label_colours[ll, 0]
        g[label_mask == ll] = label_colours[ll, 1]
        b[label_mask == ll] = label_colours[ll, 2]
    
    r[label_mask == 255] = 0
    g[label_mask == 255] = 0
    b[label_mask == 255] =0
    
#    rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
#   # rgb = np.zeros((label_mask.shape[1], label_mask.shape[2], 3))
#    #replace blue with red as opencv uses bgr
#    rgb[:, :, 0] = r /255.0     
#    rgb[:, :, 1] = g /255.0
#    rgb[:, :, 2] = b /255.0
#    
    rgb = np.zeros((label_mask.shape[1], label_mask.shape[2], 3))
    #replace blue with red as opencv uses bgr
    rgb[:, :, 0] = b #/255.0     
    rgb[:, :, 1] = g #/255.0
    rgb[:, :, 2] = r #/255.0
#    
    if plot:
        plt.imshow(rgb)
        plt.show()
    else:
        return rgb
def generate_param_report(logfile, param):
    log_file = open(logfile, 'w')
    for key, val in param.items():
        log_file.write(key + ':' + str(val) + '\n')
    log_file.close()


def lr_poly(base_lr, iter_, max_iter=100, power=0.9):
    return base_lr * ((1 - float(iter_) / max_iter) ** power)


    
from torchvision import transforms 

if __name__ == '__main__':
    print()
    ar=np.array([[0,7,10],[7,3,6]])
    z=convertTrainIdToClassId(ar)
#    img3= transforms.ToPILImage()(torch.from_numpy(ou).type(torch.FloatTensor))#.detach().cpu()
#    img3.save(oupath)
    print(z)


[[ 7 20 23]
 [20 12 19]]


# data loader

## Cityscapes code
- LiteSeg-master/dataloaders/cityscapes.py

In [8]:

#from torch.utils.data import TensorDataset, DataLoader
#from dataloaders.utils import listFiles #모르겠음

class Cityscapes(data.Dataset):

    def __init__(self, root='path', split="train", transform=None,extra=False):
        """
        Cityscapes dataset folder has two folders, 'leftImg8bit' folder for images and 'gtFine_trainvaltest' 
        folder for annotated images with fine annotations 'labels'.
        """
        self.root = root
        self.split = split #train, validation, and test sets
        self.transform = transform
        self.files = {}
        self.n_classes = 19
        self.extra=extra

        if not self.extra:
            print("Using fine dataset")
            self.images_path = os.path.join(self.root, 'leftImg8bit_trainvaltest','leftImg8bit', self.split) #실제이미지
            self.labels_path = os.path.join(self.root, 'gtFine_trainvaltest', 'gtFine', self.split) #세그멘테이션 이미지
        else:
            print("Using Coarse dataset")

            self.images_path = os.path.join(self.root, 'leftImg8bit', self.split)
            self.labels_path = os.path.join(self.root, 'gtCoarse', 'gtCoarse', self.split)            
            
        #print(self.images_path)
        self.files[split] = listFiles(rootdir=self.images_path, suffix='.png')#list of the pathes to images

        self.void_classes = [0, 1, 2, 3, 4, 5, 6, 9, 10, 14, 15, 16, 18, 29, 30, -1] #not to train
        self.valid_classes = [7, 8, 11, 12, 13, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33]
        self.class_names = ['road', 'sidewalk', 'building', 'wall', 'fence', \
                            'pole', 'traffic_light', 'traffic_sign', 'vegetation', 'terrain', \
                            'sky', 'person', 'rider', 'car', 'truck', 'bus', 'train', \
                            'motorcycle', 'bicycle']
        
        self.ignore_index = 255
        self.class_map = dict(zip(self.valid_classes, range(self.n_classes)))
        #print(self.class_map)
        
        if not self.files[split]:
            raise Exception("No files for split=[%s] found in %s" % (split, self.images.path))

        print("Found %d %s images" % (len(self.files[split]), split))
        
    
    def __len__(self):
        return len(self.files[self.split])
    
    def __getitem__(self, index):
        image_path = self.files[self.split][index].rstrip()
        #print(image_path)
        if not self.extra:
            label_path = os.path.join(self.labels_path,
                                image_path.split(os.sep)[-2],
                                os.path.basename(image_path)[:-15] + 'gtFine_labelIds.png')
        else:
            label_path = os.path.join(self.labels_path,
                                image_path.split(os.sep)[-2],
                                os.path.basename(image_path)[:-15] + 'gtCoarse_labelIds.png')
        _img = Image.open(image_path).convert('RGB')
        _tmp = np.array(Image.open(label_path), dtype=np.uint8)
        _tmp = self.encode_segmap(_tmp)

        _target = Image.fromarray(_tmp)

        sample = {'image': _img, 'label': _target}

        if self.transform:
            sample = self.transform(sample)
        return sample
    
    def encode_segmap(self, mask):
        # Put all void classes to ignore_index
        for _voidc in self.void_classes:
            mask[mask == _voidc] = self.ignore_index
        for _validc in self.valid_classes:
            mask[mask == _validc] = self.class_map[_validc]
        return mask

print("ok")

ok


## Augmentation code
- LiteSeg-master/dataloaders/augmentation.py

In [9]:
import torch
import math
import numbers
import random
import numpy as np

from PIL import Image, ImageOps

class RandomCrop(object):
    def __init__(self, size, padding=0):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size # h, w
        self.padding = padding

    def __call__(self, sample):
        img, mask = sample['image'], sample['label']

        if self.padding > 0:
            img = ImageOps.expand(img, border=self.padding, fill=255)
            mask = ImageOps.expand(mask, border=self.padding, fill=255)

        assert img.size == mask.size
        w, h = img.size
        th, tw = self.size # target size
        if w == tw and h == th:
            return {'image': img,
                    'label': mask}
        if w < tw or h < th:
            img = img.resize((tw, th), Image.BILINEAR)
            mask = mask.resize((tw, th), Image.NEAREST)
            return {'image': img,
                    'label': mask}

        x1 = random.randint(0, w - tw)
        y1 = random.randint(0, h - th)
        img = img.crop((x1, y1, x1 + tw, y1 + th))
        mask = mask.crop((x1, y1, x1 + tw, y1 + th))

        return {'image': img,
                'label': mask}


class CenterCrop(object):
    def __init__(self, size):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size

    def __call__(self, sample):
        img = sample['image']
        mask = sample['label']
        assert img.size == mask.size
        w, h = img.size
        th, tw = self.size
        x1 = int(round((w - tw) / 2.))
        y1 = int(round((h - th) / 2.))
        img = img.crop((x1, y1, x1 + tw, y1 + th))
        mask = mask.crop((x1, y1, x1 + tw, y1 + th))

        return {'image': img,
                'label': mask}


class RandomHorizontalFlip(object):
    def __call__(self, sample):
        img = sample['image']
        mask = sample['label']
        if random.random() < 0.5:
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
            mask = mask.transpose(Image.FLIP_LEFT_RIGHT)

        return {'image': img,
                'label': mask}


class Normalize(object):
    """Normalize a tensor image with mean and standard deviation.
    Args:
        mean (tuple): means for each channel.
        std (tuple): standard deviations for each channel.
    """
    def __init__(self, mean=(0., 0., 0.), std=(1., 1., 1.)):
        self.mean = mean
        self.std = std

    def __call__(self, sample):
        img = np.array(sample['image']).astype(np.float32)
        mask = np.array(sample['label']).astype(np.float32)
        img /= 255.0
        img -= self.mean
        img /= self.std

        return {'image': img,
                'label': mask}


class Normalize_cityscapes(object):
    """Normalize a tensor image with mean and standard deviation.
    Args:
        mean (tuple): means for each channel.
        std (tuple): standard deviations for each channel.
    """
    def __init__(self, mean=(0., 0., 0.)):
        self.mean = mean

    def __call__(self, sample):
        img = np.array(sample['image']).astype(np.float32)
        mask = np.array(sample['label']).astype(np.float32)
        img -= self.mean
        img /= 255.0

        return {'image': img,
                'label': mask}

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        img = np.array(sample['image']).astype(np.float32).transpose((2, 0, 1))
        mask = np.expand_dims(np.array(sample['label']).astype(np.float32), -1).transpose((2, 0, 1))
#        mask[mask == 255] = 19

        img = torch.from_numpy(img).float()
        mask = torch.from_numpy(mask).float()
#        print("MAsk from Custom transforsms"+str(mask.size()))

        return {'image': img,
                'label': mask}


class FixedResize(object):
    def __init__(self, size):
        self.size = tuple(reversed(size))  # size: (h, w)

    def __call__(self, sample):
        img = sample['image']
        mask = sample['label']

        assert img.size == mask.size

        img = img.resize(self.size, Image.BILINEAR)
        mask = mask.resize(self.size, Image.NEAREST)

        return {'image': img,
                'label': mask}


class Scale(object):
    def __init__(self, size):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size

    def __call__(self, sample):
        img = sample['image']
        mask = sample['label']
        assert img.size == mask.size
        w, h = img.size

        if (w >= h and w == self.size[1]) or (h >= w and h == self.size[0]):
            return {'image': img,
                    'label': mask}
        oh, ow = self.size
        img = img.resize((ow, oh), Image.BILINEAR)
        mask = mask.resize((ow, oh), Image.NEAREST)

        return {'image': img,
                'label': mask}


class RandomSizedCrop(object):
    def __init__(self, size):
        self.size = size

    def __call__(self, sample):
        img = sample['image']
        mask = sample['label']
        assert img.size == mask.size
        for attempt in range(10):
            area = img.size[0] * img.size[1]
            target_area = random.uniform(0.45, 1.0) * area
            aspect_ratio = random.uniform(0.5, 2)

            w = int(round(math.sqrt(target_area * aspect_ratio)))
            h = int(round(math.sqrt(target_area / aspect_ratio)))

            if random.random() < 0.5:
                w, h = h, w

            if w <= img.size[0] and h <= img.size[1]:
                x1 = random.randint(0, img.size[0] - w)
                y1 = random.randint(0, img.size[1] - h)

                img = img.crop((x1, y1, x1 + w, y1 + h))
                mask = mask.crop((x1, y1, x1 + w, y1 + h))
                assert (img.size == (w, h))

                img = img.resize((self.size, self.size), Image.BILINEAR)
                mask = mask.resize((self.size, self.size), Image.NEAREST)

                return {'image': img,
                        'label': mask}

        # Fallback
        scale = Scale(self.size)
        crop = CenterCrop(self.size)
        sample = crop(scale(sample))
        return sample


class RandomRotate(object):
    def __init__(self, degree):
        self.degree = degree

    def __call__(self, sample):
        img = sample['image']
        mask = sample['label']
        rotate_degree = random.random() * 2 * self.degree - self.degree
        img = img.rotate(rotate_degree, Image.BILINEAR)
        #mask = mask.rotate(rotate_degree, Image.NEAREST)
        im2 = mask.convert('RGBA')
        # rotated image
        rot = im2.rotate(rotate_degree, Image.NEAREST)
        # a white image same size as rotated image
        fff = Image.new('RGBA', rot.size, (255,)*4)
        # create a composite image using the alpha layer of rot as a mask
        out = Image.composite(rot, fff, rot)
        # save your work (converting back to mode='1' or whatever..)
        out=out.convert(mask.mode)

        return {'image': img,
                'label': out}


class RandomSized(object):
    def __init__(self, size):
        self.size = size
        self.scale = Scale(self.size)
        self.crop = RandomCrop(self.size)

    def __call__(self, sample):
        img = sample['image']
        mask = sample['label']
        assert img.size == mask.size

        w = int(random.uniform(0.8, 2.5) * img.size[0])
        h = int(random.uniform(0.8, 2.5) * img.size[1])

        img, mask = img.resize((w, h), Image.BILINEAR), mask.resize((w, h), Image.NEAREST)
        sample = {'image': img, 'label': mask}

        return self.crop(self.scale(sample))
class RandomScaleCrop(object):
    def __init__(self, base_size, crop_size, fill=255):
        self.base_size = base_size
        self.crop_size = crop_size
        self.fill = fill

    def __call__(self, sample):
        img = sample['image']
        mask = sample['label']
        # random scale (short edge)
        short_size = random.randint(int(self.base_size * 0.5), int(self.base_size * 2.0))
        w, h = img.size
        if h > w:
            ow = short_size
            oh = int(1.0 * h * ow / w)
        else:
            oh = short_size
            ow = int(1.0 * w * oh / h)
        img = img.resize((ow, oh), Image.BILINEAR)
        mask = mask.resize((ow, oh), Image.NEAREST)
        # pad crop
        if short_size < self.crop_size:
            padh = self.crop_size - oh if oh < self.crop_size else 0
            padw = self.crop_size - ow if ow < self.crop_size else 0
            img = ImageOps.expand(img, border=(0, 0, padw, padh), fill=0)
            mask = ImageOps.expand(mask, border=(0, 0, padw, padh), fill=self.fill)
        # random crop crop_size
        w, h = img.size
        x1 = random.randint(0, w - self.crop_size)
        y1 = random.randint(0, h - self.crop_size)
        img = img.crop((x1, y1, x1 + self.crop_size, y1 + self.crop_size))
        mask = mask.crop((x1, y1, x1 + self.crop_size, y1 + self.crop_size))

        return {'image': img,
                'label': mask}
        
class RandomScale(object):
    def __init__(self, limit):
        self.limit = limit

    def __call__(self, sample):
        img = sample['image']
        mask = sample['label']
        assert img.size == mask.size

        scale = random.uniform(self.limit[0], self.limit[1])
        w = int(scale * img.size[0])
        h = int(scale * img.size[1])

        img, mask = img.resize((w, h), Image.BILINEAR), mask.resize((w, h), Image.NEAREST)

        return {'image': img, 'label': mask}

In [10]:
from collections import OrderedDict
import lightnet.network as lnn

/home/aiffel-dj17/anaconda3/envs/sia/lib/python3.7/site-packages/scipy/__init__.py:140: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.15.4)
  UserWarning)


## LiteSeg-master/models/backbone_networks/darknet.py
- git = pytorch 0.4.1

In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 19 21:47:01 2018

@author: Taha Emara  @email: taha@emaraic.com 'based on lightnet library by EAVISE' 
"""

from collections import OrderedDict
import torch.nn as nn
import lightnet.network as lnn



class Darknet19(lnn.module.Darknet):
    """ `Darknet19`_ implementation with pytorch.

    Args:
        weights_file (str, optional): Path to the saved weights; Default **None**
        input_channels (Number, optional): Number of input channels; Default **3**

    .. _Darknet19: https://github.com/pjreddie/darknet/blob/master/cfg/darknet19.cfg
    """

    def __init__(self, weights_file=None, input_channels=3):
        """ Network initialisation """
        super().__init__()


        # Network
        self.layers = nn.Sequential(
            OrderedDict([
                ('1_convbatch',     lnn.layer.Conv2dBatchReLU(input_channels, 32, 3, 1, 1)),
                ('2_max',           nn.MaxPool2d(2, 2)),
                ('3_convbatch',     lnn.layer.Conv2dBatchReLU(32, 64, 3, 1, 1)),
                ('4_max',           nn.MaxPool2d(2, 2)),
                ('5_convbatch',     lnn.layer.Conv2dBatchReLU(64, 128, 3, 1, 1)),
                ('6_convbatch',     lnn.layer.Conv2dBatchReLU(128, 64, 1, 1, 0)),
                ('7_convbatch',     lnn.layer.Conv2dBatchReLU(64, 128, 3, 1, 1)),
                ('8_max',           nn.MaxPool2d(2,2)),
                ('9_convbatch',     lnn.layer.Conv2dBatchReLU(128, 256, 3, 1, 1)),#2>1
                ('10_convbatch',    lnn.layer.Conv2dBatchReLU(256, 128, 1, 1, 0)),
                ('11_convbatch',    lnn.layer.Conv2dBatchReLU(128, 256, 3, 1, 1)),
                ('12_max',          nn.MaxPool2d(2,2)),
                ('13_convbatch',    lnn.layer.Conv2dBatchReLU(256, 512, 3, 1, 1,dilation=1)),
                ('14_convbatch',    lnn.layer.Conv2dBatchReLU(512, 256, 1,1, 1,dilation=1)),
                ('15_convbatch',    lnn.layer.Conv2dBatchReLU(256, 512, 3, 1, 1,dilation=1)),
                ('16_convbatch',    lnn.layer.Conv2dBatchReLU(512, 256, 1, 1, 1,dilation=1)),
                ('17_convbatch',    lnn.layer.Conv2dBatchReLU(256, 512, 3, 1, 1,dilation=1)),
                ('18_max',          nn.MaxPool2d(1,1)),
                ('19_convbatch',    lnn.layer.Conv2dBatchReLU(512, 1024, 3, 1, 1,dilation=2)),
                ('20_convbatch',    lnn.layer.Conv2dBatchReLU(1024, 512, 1, 1, 1,dilation=2)),
                ('21_convbatch',    lnn.layer.Conv2dBatchReLU(512, 1024, 3, 1, 1,dilation=2)),
                ('22_convbatch',    lnn.layer.Conv2dBatchReLU(1024, 512, 1, 1, 0,dilation=2)),
                ('23_convbatch',    lnn.layer.Conv2dBatchReLU(512, 1024, 3, 1, 1,dilation=2))
            ])
        )

        if weights_file is not None:
            self.load(weights_file)

    def _forward(self, x):
        i=0
        for i,l in enumerate( self.layers[:7]):
            #print(str(i),x.size())
            x = l(x)
        keep = x
        for z,l in enumerate(self.layers[7:]):
            #print(str(z+i),x.size())
            x = l(x)
        return x,keep

""" os =16
   ('1_convbatch',     lnn.layer.Conv2dBatchReLU(input_channels, 32, 3, 1, 1)),
                ('2_max',           nn.MaxPool2d(2, 2)),
                ('3_convbatch',     lnn.layer.Conv2dBatchReLU(32, 64, 3, 1, 1)),
                ('4_max',           nn.MaxPool2d(2, 2)),
                ('5_convbatch',     lnn.layer.Conv2dBatchReLU(64, 128, 3, 1, 1)),
                ('6_convbatch',     lnn.layer.Conv2dBatchReLU(128, 64, 1, 1, 0)),
                ('7_convbatch',     lnn.layer.Conv2dBatchReLU(64, 128, 3, 1, 1)),
                ('8_max',           nn.MaxPool2d(2,2)),
                ('9_convbatch',     lnn.layer.Conv2dBatchReLU(128, 256, 3, 1, 1)),#2>1
                ('10_convbatch',    lnn.layer.Conv2dBatchReLU(256, 128, 1, 1, 0)),
                ('11_convbatch',    lnn.layer.Conv2dBatchReLU(128, 256, 3, 1, 1)),
                ('12_max',          nn.MaxPool2d(2, 2)),
                ('13_convbatch',    lnn.layer.Conv2dBatchReLU(256, 512, 3, 1, 1,dilation=1)),
                ('14_convbatch',    lnn.layer.Conv2dBatchReLU(512, 256, 1,1, 1,dilation=1)),
                ('15_convbatch',    lnn.layer.Conv2dBatchReLU(256, 512, 3, 1, 1,dilation=1)),
                ('16_convbatch',    lnn.layer.Conv2dBatchReLU(512, 256, 1, 1, 1,dilation=1)),
                ('17_convbatch',    lnn.layer.Conv2dBatchReLU(256, 512, 3, 1, 1,dilation=1)),
                ('18_max',          nn.MaxPool2d(1,1)),
                ('19_convbatch',    lnn.layer.Conv2dBatchReLU(512, 1024, 3, 1, 1,dilation=2)),
                ('20_convbatch',    lnn.layer.Conv2dBatchReLU(1024, 512, 1, 1, 1,dilation=2)),
                ('21_convbatch',    lnn.layer.Conv2dBatchReLU(512, 1024, 3, 1, 1,dilation=2)),
                ('22_convbatch',    lnn.layer.Conv2dBatchReLU(1024, 512, 1, 1, 0,dilation=2)),
                ('23_convbatch',    lnn.layer.Conv2dBatchReLU(512, 1024, 3, 1, 1,dilation=2))
                
"""
"""os =32
 ('1_convbatch',     lnn.layer.Conv2dBatchReLU(input_channels, 32, 3, 1, 1)),
                ('2_max',           nn.MaxPool2d(2, 2)),
                ('3_convbatch',     lnn.layer.Conv2dBatchReLU(32, 64, 3, 1, 1)),
                ('4_max',           nn.MaxPool2d(2, 2)),
                ('5_convbatch',     lnn.layer.Conv2dBatchReLU(64, 128, 3, 1, 1)),
                ('6_convbatch',     lnn.layer.Conv2dBatchReLU(128, 64, 1, 1, 0)),
                ('7_convbatch',     lnn.layer.Conv2dBatchReLU(64, 128, 3, 1, 1)),
                ('8_max',           nn.MaxPool2d(2,2)),
                ('9_convbatch',     lnn.layer.Conv2dBatchReLU(128, 256, 3, 1, 1)),#2>1
                ('10_convbatch',    lnn.layer.Conv2dBatchReLU(256, 128, 1, 1, 0)),
                ('11_convbatch',    lnn.layer.Conv2dBatchReLU(128, 256, 3, 1, 1)),
                ('12_max',          nn.MaxPool2d(2, 2)),
                ('13_convbatch',    lnn.layer.Conv2dBatchReLU(256, 512, 3, 1, 1,dilation=2)),
                ('14_convbatch',    lnn.layer.Conv2dBatchReLU(512, 256, 1,1, 0,dilation=2)),
                ('15_convbatch',    lnn.layer.Conv2dBatchReLU(256, 512, 3, 1, 1,dilation=2)),
                ('16_convbatch',    lnn.layer.Conv2dBatchReLU(512, 256, 1, 1, 0,dilation=2)),
                ('17_convbatch',    lnn.layer.Conv2dBatchReLU(256, 512, 3, 1, 1,dilation=2)),
                ('18_max',          nn.MaxPool2d(1,1)),
                ('19_convbatch',    lnn.layer.Conv2dBatchReLU(512, 1024, 3, 1, 1,dilation=4)),
                ('20_convbatch',    lnn.layer.Conv2dBatchReLU(1024, 512, 1, 1, 0,dilation=4)),
                ('21_convbatch',    lnn.layer.Conv2dBatchReLU(512, 1024, 3, 1, 1,dilation=4)),
                ('22_convbatch',    lnn.layer.Conv2dBatchReLU(1024, 512, 1, 1, 0,dilation=4)),
                ('23_convbatch',    lnn.layer.Conv2dBatchReLU(512, 1024, 3, 1, 1,dilation=4))
"""


"os =32\n ('1_convbatch',     lnn.layer.Conv2dBatchReLU(input_channels, 32, 3, 1, 1)),\n                ('2_max',           nn.MaxPool2d(2, 2)),\n                ('3_convbatch',     lnn.layer.Conv2dBatchReLU(32, 64, 3, 1, 1)),\n                ('4_max',           nn.MaxPool2d(2, 2)),\n                ('5_convbatch',     lnn.layer.Conv2dBatchReLU(64, 128, 3, 1, 1)),\n                ('6_convbatch',     lnn.layer.Conv2dBatchReLU(128, 64, 1, 1, 0)),\n                ('7_convbatch',     lnn.layer.Conv2dBatchReLU(64, 128, 3, 1, 1)),\n                ('8_max',           nn.MaxPool2d(2,2)),\n                ('9_convbatch',     lnn.layer.Conv2dBatchReLU(128, 256, 3, 1, 1)),#2>1\n                ('10_convbatch',    lnn.layer.Conv2dBatchReLU(256, 128, 1, 1, 0)),\n                ('11_convbatch',    lnn.layer.Conv2dBatchReLU(128, 256, 3, 1, 1)),\n                ('12_max',          nn.MaxPool2d(2, 2)),\n                ('13_convbatch',    lnn.layer.Conv2dBatchReLU(256, 512, 3, 1, 1,dilation=2)),\

## LiteSeg-master/models/aspp.py

In [12]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 19 18:02:59 2018

@author: Taha Emara  @email: taha@emaraic.com
"""
import torch.nn as nn
import torch
class SeparableConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
        super(SeparableConv2d,self).__init__()

        self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size,stride,padding,dilation,groups=in_channels,bias=bias)
        self.pointwise = nn.Conv2d(in_channels,out_channels,1,1,0,1,1,bias=bias)

    def forward(self,x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x
class ASPP(nn.Module):
    
    def __init__(self, inplanes, planes, rate):
        super(ASPP, self).__init__()
        self.rate=rate
        if rate == 1:
            kernel_size = 1
            padding = 0
        else:
            kernel_size = 3
            padding = rate
            #self.conv1 = nn.Conv2d(planes, planes, kernel_size=3, bias=False,padding=1)
            self.conv1 =SeparableConv2d(planes,planes,3,1,1)
            self.bn1 = nn.BatchNorm2d(planes)
            self.relu1 = nn.ReLU()
   
            #self.atrous_convolution = nn.Conv2d(inplanes, planes, kernel_size=kernel_size,
            #                         stride=1, padding=padding, dilation=rate, bias=False)
        self.atrous_convolution = SeparableConv2d(inplanes,planes,kernel_size,1,padding,rate)
        self.bn = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU()
        
        
        self._init_weight()

    def forward(self, x):
        x = self.atrous_convolution(x)
        x = self.bn(x)
        #x = self.relu(x)
        if self.rate!=1:
            x=self.conv1(x)
            x=self.bn1(x)
            x=self.relu1(x)
        return x

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

## LiteSeg-master/models/separableconv.py

In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jan 13 11:05:01 2019

@author: Taha Emara  @email: taha@emaraic.com
"""
import torch.nn as nn


class SeparableConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
        super(SeparableConv2d,self).__init__()

        self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size,stride,padding,dilation,groups=in_channels,bias=bias)
        self.pointwise = nn.Conv2d(in_channels,out_channels,1,1,0,1,1,bias=bias)

    def forward(self,x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x

## LiteSeg-master/models/liteseg_darknet.py

In [14]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 19 22:20:16 2018

@author: Taha Emara  @email: taha@emaraic.com
"""

import torch.nn as nn
import torch.nn.functional as F
import torch
import math


#from models.backbone_networks import darknet
#from models import aspp
#from models.separableconv import SeparableConv2d 

# 클래스명 RT -> Darknet_RT로 수정
# super(RT, -> super(Darknet_RT, 로 수정

class Darknet_RT(nn.Module):
    
    def __init__(self,  n_classes=19, pretrained=True,PRETRAINED_WEIGHTS="."):
       
        super(Darknet_RT, self).__init__()
        print("LiteSeg-DarkNet...")
        if pretrained:
            self.resnet_features=Darknet19(weights_file=PRETRAINED_WEIGHTS)
            print('ooooooooooooooooo')
        else:
            self.resnet_features=Darknet19(weights_file=None)
            print('xxxxxxxxxxxxxxxxx')
        rates = [1, 3, 6, 9]

        self.aspp1 = ASPP(1024, 96, rate=rates[0])
        self.aspp2 = ASPP(1024, 96, rate=rates[1])
        self.aspp3 = ASPP(1024, 96, rate=rates[2])
        self.aspp4 = ASPP(1024, 96, rate=rates[3])

        self.relu = nn.ReLU()
        self.global_avg_pool = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                                             nn.Conv2d(1024, 96, 1, stride=1, bias=False),
                                             nn.BatchNorm2d(96),
                                             nn.ReLU())
        
        #self.conv1 = nn.Conv2d(1504, 96, 1, bias=False)#480 1504
        self.conv1 =SeparableConv2d(1504,96,1)
        self.bn1 = nn.BatchNorm2d(96)

        # adopt [1x1, 48] for channel reduction.
        #self.conv2 = nn.Conv2d(128, 32, 1, bias=False)#128 for no previous feature 7  ,64 ---3
        self.conv2 = SeparableConv2d(128,32,1)
        self.bn2 = nn.BatchNorm2d(32)
#    
        self.last_conv = nn.Sequential(#nn.Conv2d(128, 96, kernel_size=3, stride=1, padding=1, bias=False),
                                       SeparableConv2d(128,96,3,1,1),
                                       nn.BatchNorm2d(96),
                                       nn.ReLU(),
                                       #nn.Conv2d(96, 96, kernel_size=3, stride=1, padding=1, bias=False),
                                       SeparableConv2d(96,96,3,1,1),
                                       nn.BatchNorm2d(96),
                                       nn.ReLU(),
                                       nn.Conv2d(96, n_classes, kernel_size=1, stride=1))
 
    def forward(self, input):
        x, low_level_features = self.resnet_features(input)
        #print('x ',x.size(),' low features',low_level_features.size())
        x1 = self.aspp1(x)
        x2 = self.aspp2(x)
        x3 = self.aspp3(x)
        x4 = self.aspp4(x)
        x5 = self.global_avg_pool(x)
        x5 = F.interpolate(x5, size=x4.size()[2:], mode='bilinear', align_corners=True)
        #print('x1',x1.size())
        x = torch.cat((x,x1, x2, x3, x4, x5), dim=1)#x = torch.cat((x,x1, x2, x3, x4, x5), dim=1)

        #ablation=torch.max(x, 1)[1]

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = F.interpolate(x, size=(int(math.ceil(input.size()[-2]/4)),##/4 --7  /2 ---3
                                int(math.ceil(input.size()[-1]/4))), mode='bilinear', align_corners=True)
        #ablation=torch.max(x, 1)[1]
        #print('x',x.size())

        ##comment to remove low feature
        #print('low level features',low_level_features.size())
        low_level_features = self.conv2(low_level_features)
        low_level_features = self.bn2(low_level_features)
        low_level_features = self.relu(low_level_features)
        x = torch.cat((x, low_level_features), dim=1)

        
        #ablation=torch.max(x, 1)[1]
        x = self.last_conv(x)
        x = F.interpolate(x, size=input.size()[2:], mode='bilinear', align_corners=True)

        return x #,ablation

    def freeze_bn(self):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    def __init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


## siaiffel/LiteSeg-master/models/bacbone_networks/MobileNetV2.py

In [15]:

"""
Created on Mon Nov 19 21:47:01 2018

@author: https://github.com/ericsun99/MobileNet-V2-Pytorch, edited by:Taha Emara  @email: taha@emaraic.com
"""

import torch.nn as nn
import math

def conv_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.use_res_connect = self.stride == 1 and inp == oup

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(self, n_class=1000, input_size=224, width_mult=1.):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 1280
        interverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # building first layer
        assert input_size % 32 == 0
        input_channel = int(input_channel * width_mult)
        self.last_channel = int(last_channel * width_mult) if width_mult > 1.0 else last_channel
        self.features = [conv_bn(3, input_channel, 2)]
        # building inverted residual blocks
        for t, c, n, s in interverted_residual_setting:
            output_channel = int(c * width_mult)
            for i in range(n):
                if i == 0:
                    self.features.append(block(input_channel, output_channel, s, expand_ratio=t))
                else:
                    self.features.append(block(input_channel, output_channel, 1, expand_ratio=t))
                input_channel = output_channel
        # building last several layers
        self.features.append(conv_1x1_bn(input_channel, self.last_channel))
        # make it nn.Sequential
        self.features = nn.Sequential(*self.features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, n_class),
        )

        self._initialize_weights()

    def forward(self, x):
        i=0
        for i,l in enumerate( self.features[:4]):
            #print(str(i),x.size())
            x = l(x)
        keep = x
        for z,l in enumerate(self.features[4:]):
            #print(str(z+i+1),x.size())
            x = l(x)
        return x,keep

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

## siaiffel/LiteSeg-master/models/liteseg_mobilenet.py

In [16]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Dec 16 11:20:32 2018

@author: Taha Emara  @email: taha@emaraic.com
"""
import torch.nn as nn
import torch.nn.functional as F
import torch
import math

#from models.backbone_networks import MobileNetV2
#from models import aspp
#from models.separableconv import SeparableConv2d 



# 클래스명 RT -> MobileNet_RT로 수정
# super(RT, -> super(MobileNet_RT, 로 수정
class MobileNet_RT(nn.Module):
    
    def __init__(self, n_classes=19,PRETRAINED_WEIGHTS=".", pretrained=True):
        
        super(MobileNet_RT, self).__init__()
        print("LiteSeg-MobileNet...")

        self.mobile_features=MobileNetV2()
        if pretrained:
            state_dict = torch.load(PRETRAINED_WEIGHTS)
            self.mobile_features.load_state_dict(state_dict)
        
        rates = [1, 3, 6, 9]


        self.aspp1 = ASPP(1280, 96, rate=rates[0])
        self.aspp2 = ASPP(1280, 96, rate=rates[1])
        self.aspp3 = ASPP(1280, 96, rate=rates[2])
        self.aspp4 = ASPP(1280, 96, rate=rates[3])

        self.relu = nn.ReLU()
        self.global_avg_pool = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                                             nn.Conv2d(1280, 96, 1, stride=1, bias=False),
                                             nn.BatchNorm2d(96),
                                             nn.ReLU())
        #self.conv1 = nn.Conv2d(480+1280, 96, 1, bias=False)
        self.conv1 =SeparableConv2d(480+1280,96,1)
        self.bn1 = nn.BatchNorm2d(96)

        #adopt [1x1, 48] for channel reduction.
        #self.conv2 = nn.Conv2d(24, 32, 1, bias=False)
        #self.bn2 = nn.BatchNorm2d(32)
    
        self.last_conv = nn.Sequential(#nn.Conv2d(24+96, 96, kernel_size=3, stride=1, padding=1, bias=False),
                                       SeparableConv2d(24+96,96,3,1,1),
                                       nn.BatchNorm2d(96),
                                       nn.ReLU(),
                                       #nn.Conv2d(96, 96, kernel_size=3, stride=1, padding=1, bias=False),
                                       SeparableConv2d(96,96,3,1,1),
                                       nn.BatchNorm2d(96),
                                       nn.ReLU(),
                                       nn.Conv2d(96, n_classes, kernel_size=1, stride=1))
        
    def forward(self, input):
        x, low_level_features = self.mobile_features(input)
        #print(x.size())
        x1 = self.aspp1(x)
        x2 = self.aspp2(x)
        x3 = self.aspp3(x)
        x4 = self.aspp4(x)
        x5 = self.global_avg_pool(x)
        x5 = F.interpolate(x5, size=x4.size()[2:], mode='bilinear', align_corners=True)

        x = torch.cat((x,x1, x2, x3, x4, x5), dim=1)
        #print('after aspp cat',x.size())
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = F.interpolate(x, size=(int(math.ceil(input.size()[-2]/4)),
                                int(math.ceil(input.size()[-1]/4))), mode='bilinear', align_corners=True)
       # ablation=torch.max(low_level_features, 1)[1]
        #print('after con on aspp output',x.size())

        ##comment to remove low feature
        #low_level_features = self.conv2(low_level_features)
        #low_level_features = self.bn2(low_level_features)
        #low_level_features = self.relu(low_level_features)
        #print("low",low_level_features.size())
        
        x = torch.cat((x, low_level_features), dim=1)
        #print('after cat low feature with output of aspp',x.size())

        x = self.last_conv(x)
        x = F.interpolate(x, size=input.size()[2:], mode='bilinear', align_corners=True)

        return x#,ablation

    def freeze_bn(self):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    def __init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

## siaiffel/LiteSeg-master/models/liteseg.py

In [17]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jan 13 12:00:52 2019

@author: Taha Emara  @email: taha@emaraic.com
"""
import torch 

#from models import liteseg_shufflenet as shufflenet
#from models import liteseg_darknet as darknet
#from models import liteseg_mobilenet as mobilenet



class LiteSeg():
    
    #수정 사항 : Darknet_RT, MobileNet_RT 
    def build(backbone_network,modelpath,CONFIG,is_train=True):
                
        if backbone_network.lower() == 'darknet':
            net = Darknet_RT(n_classes=19, pretrained=is_train,PRETRAINED_WEIGHTS=CONFIG.PRETRAINED_DarkNET19)
        elif backbone_network.lower() == 'shufflenet':
            net = shufflenet.RT(n_classes=19, pretrained=is_train, PRETRAINED_WEIGHTS=CONFIG.PRETRAINED_SHUFFLENET)
        elif backbone_network.lower() == 'mobilenet':
            net = MobileNet_RT(n_classes=19,pretrained=is_train, PRETRAINED_WEIGHTS=CONFIG.PRETRAINED_MOBILENET)
        else:
            raise NotImplementedError
            
        if modelpath is not None:
            net.load_state_dict(torch.load(modelpath))
            
        print("Using LiteSeg with",backbone_network)
        return net
        
            

## siaiffel/LiteSeg-master/utils/loss.py

In [18]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 31 17:52:22 2018

@author: Taha Emara  @email: taha@emaraic.com
"""
import torch.nn as nn

def cross_entropy2d(logit, target, ignore_index=255, weight=None, reduct='elementwise_mean'):
    n, c, h, w = logit.size()
    #print("Inside Cross entropy ",logit.size()  ,"Target Size",target.size())
    # logit = logit.permute(0, 2, 3, 1)
    target = target.squeeze(1)
    if weight is None:
        criterion = nn.CrossEntropyLoss(ignore_index=ignore_index,reduction=reduct)
    else:
        criterion = nn.CrossEntropyLoss(weight=weight, ignore_index=ignore_index,reduction=reduct)
    loss = criterion(logit, target.long())

    batch_average=True
    if batch_average:
        loss /= n

    return loss

## siaiffel/LiteSeg-master/utils/iou_eval.py

In [19]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 31 18:28:07 2018

@author: Taha Emara  @email: taha@emaraic.com
"""

import torch 

class Eval:

    def __init__(self, nClasses, ignoreIndex=19):
        self.nClasses = nClasses
        self.ignoreIndex = ignoreIndex if nClasses>ignoreIndex else -1 #if ignoreIndex is larger than nClasses, consider no ignoreIndex
        self.reset()

    def reset (self):
        classes = self.nClasses if self.ignoreIndex==-1 else self.nClasses-1
        self.tp = torch.zeros(classes).double()
        self.fp = torch.zeros(classes).double()
        self.fn = torch.zeros(classes).double()        

    def addBatch(self, x, y):   #x=preds, y=targets
        #sizes should be "batch_size x nClasses x H x W"
        
        #print ("X is cuda: ", x.is_cuda)
        #print ("Y is cuda: ", y.is_cuda)
        x=x.type( torch.LongTensor)
        y=y.type( torch.LongTensor)
        if (x.is_cuda or y.is_cuda):
            x = x.cuda()
            y = y.cuda()

        #if size is "batch_size x 1 x H x W" scatter to onehot
        if (x.size(1) == 1):
            x_onehot = torch.zeros(x.size(0), self.nClasses, x.size(2), x.size(3))  
            if x.is_cuda:
                x_onehot = x_onehot.cuda()
            x_onehot.scatter_(1, x, 1).float()
        else:
            x_onehot = x.float()

        if (y.size(1) == 1):
            y_onehot = torch.zeros(y.size(0), self.nClasses, y.size(2), y.size(3))
            if y.is_cuda:
                y_onehot = y_onehot.cuda()
            y_onehot.scatter_(1, y, 1).float()
        else:
            y_onehot = y.float()

        if (self.ignoreIndex != -1): 
            ignores = y_onehot[:,self.ignoreIndex].unsqueeze(1)
            x_onehot = x_onehot[:, :self.ignoreIndex]
            y_onehot = y_onehot[:, :self.ignoreIndex]
        else:
            ignores=0

        #print(type(x_onehot))
        #print(type(y_onehot))
        #print(x_onehot.size())
        #print(y_onehot.size())

        tpmult = x_onehot * y_onehot    #times prediction and gt coincide is 1
        tp = torch.sum(torch.sum(torch.sum(tpmult, dim=0, keepdim=True), dim=2, keepdim=True), dim=3, keepdim=True).squeeze()
        fpmult = x_onehot * (1-y_onehot-ignores) #times prediction says its that class and gt says its not (subtracting cases when its ignore label!)
        fp = torch.sum(torch.sum(torch.sum(fpmult, dim=0, keepdim=True), dim=2, keepdim=True), dim=3, keepdim=True).squeeze()
        fnmult = (1-x_onehot) * (y_onehot) #times prediction says its not that class and gt says it is
        fn = torch.sum(torch.sum(torch.sum(fnmult, dim=0, keepdim=True), dim=2, keepdim=True), dim=3, keepdim=True).squeeze() 

        self.tp += tp.double().cpu()
        self.fp += fp.double().cpu()
        self.fn += fn.double().cpu()

    def getIoU(self):
        num = self.tp
        den = self.tp + self.fp + self.fn + 1e-15
        iou = num / den
        return torch.mean(iou), iou #returns "iou m

## train.py
- !pip install yaml
- !pip install addict
- !pip install argparse
- /home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/train.py
- python train_ms.py --backbone_network darknet --model_path_coarse ./pretrained_models/liteseg-darknet-cityscapes.pth
- weights 저장 경로 : /home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/handmade_weights
- config 경로 : /home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/config/training.yaml

## Darknet
- path 주의

In [20]:

import socket
import timeit
from datetime import datetime
import os
import glob
from collections import OrderedDict
import numpy as np
import yaml
from addict import Dict
import argparse

# PyTorch includes
import torch
from torch.autograd import Variable
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

# Tensorboard include
from tensorboardX import SummaryWriter

# Custom includes
#from dataloaders import cityscapes
#from dataloaders import utils
#from dataloaders import augmentation as augment
#from models.liteseg import LiteSeg
#from utils import loss as losses
#from utils import iou_eval


#To make reproducible results  
torch.manual_seed(125)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(125)

#경로 변경 주의
CONFIG=Dict(yaml.load(open('/home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/config/training.yaml')))

#ap, args 함수 : shell 창에서 어떤 backbone 을 쓸지, model path, pretrained_mode
#ap = argparse.ArgumentParser()
#ap.add_argument('--backbone_network', required=True,
                #help = 'name of backbone network',default='darknet')#shufflenet, mobilenet, and darknet
#ap.add_argument('--model_path_coarse', required=False,
                #help = 'path to pretrained model on coarse data',default='pretrained_models/liteseg-darknet-cityscapes.pth')
#ap.add_argument('--model_path_resume', required=False,
                #help = 'path to a model to resume from',default='pretrained_models/liteseg-darknet-cityscapes.pth')

#args = ap.parse_args()
#backbone_network= 'mobilenet'
backbone_network= 'darknet' # backbone_network 뭘로 할지 str로 입력
#model_path_resume=args.model_path_resume
#경로 변경 주의
model_path_coarse ='/home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/pretrained_models/liteseg-darknet-cityscapes.pth'
#model_path_coarse = '/home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/pretrained_models/liteseg-mobilenet-cityscapes.pth'
model_path_resume = '/home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/pretrained_models/liteseg-darknet-cityscapes.pth'


# Setting parameters
nEpochs =10  # Number of epochs for training 150 종료epoch
resume_epoch = 0  # Default is 0, change if want to resume 0 시작epoch

p = OrderedDict()  # Parameters to include in report
p['trainBatch'] =4  # Training batch size
p['lr'] =1e-7 # Learning rate  1e-8 for darknet and 1e-7 shufflenet and mobilenet
p['wd'] = 5e-4  # Weight decay
p['momentum'] = 0.9  # Momentum
p['epoch_size'] =5  # epochs to change learning rate

testBatch = 1  # Testing batch size
nValInterval = 2  # Run on test set every nTestInterval epochs
snapshot = 2  # Store a model every snapshot epochs

# 저장 경로 설정
save_dir_root = os.getenv('HOME')+'/aiffel/siaiffel/LiteSeg-master/handmade_weights'

dataset_path=CONFIG.DATASET_FINE
if CONFIG.USING_COARSE:
    print("Training on Coarse Data")
    dataset_path=CONFIG.DATASET_COARSE
    p['epoch_size'] =10 #we increase the number of epochs to change LR as we train on one scale


#txt파일 이름 설정
# exp_name = os.path.dirname(os.path.abspath(__file__)).split('/')[-1]
exp_name = 'sangmin_0520'

class_weight = np.array([0.05570516, 0.32337477, 0.08998544, 1.03602707, 1.03413147, 1.68195437,
                                 5.58540548, 3.56563995, 0.12704978, 1.,         0.46783719, 1.34551528,
                                 5.29974114, 0.28342531, 0.9396095,  0.81551811, 0.42679146, 3.6399074,
                                 2.78376194], dtype=float)
class_weight = torch.from_numpy(class_weight).float().cuda()



#make a folder -with name of current time- for every experiment
experiment_id=datetime.now().strftime("%Y-%m-%d_%H_%M")
save_path = os.path.join(save_dir_root, 'experiments', 'experiment_' + str(experiment_id))
print(save_path)


# Network definition
# LiteSeg.build 괄호 첫번째에 backbone_network 뭘로 할지 str로 입력
modelpath = model_path_coarse
net=LiteSeg.build(backbone_network, modelpath ,CONFIG, is_train=True)

if CONFIG.USING_GPU:
    torch.cuda.set_device(device=CONFIG.GPU_ID)
    net.cuda()


#using the trained model on the coarse data
#If you want to train model on fine data directley, comment the next 3 lines.
if not CONFIG.USING_COARSE:
    print("Using a weights from training coarse data from: {}...".format(model_path_coarse))
    net.load_state_dict(torch.load(model_path_coarse)) 


#resume tarining from a given model, 
#Attention! the learnig rate which used for resuming training, is not the intial one.
if resume_epoch == 0:
    print("Training Network...")
else:
    print("Resume training from a model at: {}...".format(model_path_resume))
    net.load_state_dict(torch.load(model_path_resume))

    
modelName = 'LiteSeg-' + backbone_network + '-cityscapes'
print(modelName)

criterion = cross_entropy2d

# 여기서부터 시작
if resume_epoch != nEpochs+1:
    # Logging into Tensorboard / path 설정
    log_dir = os.path.join(save_path, 'models', datetime.now().strftime('%b%d_%H-%M-%S') + '_' + socket.gethostname())
    writer = SummaryWriter(log_dir=log_dir) #summary 설정

    # Use the following optimizer 옵티마이저 설정
    optimizer = optim.SGD(net.parameters(), lr=p['lr'], momentum=p['momentum'], weight_decay=p['wd'])
    #optimizer = optim.Adam(net.parameters(), 5e-4, (0.9, 0.999), eps=1e-08, weight_decay=1e-4) 
    p['optimizer'] = str(optimizer) 

    #augment
    #augment. 삭제
    composed_transforms_tr = transforms.Compose([
        RandomHorizontalFlip(),
        RandomScale((0.2, .8)),
        RandomCrop(( 512,1024)),
        RandomRotate(5),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensor()])
   
    composed_transforms_tr1 = transforms.Compose([
        RandomHorizontalFlip(),
        RandomScale((0.2, .8)),
        RandomCrop(( 768,1536)),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensor()])
    composed_transforms_tr2 = transforms.Compose([
        RandomHorizontalFlip(),
        RandomScale((0.2, .8)),
        RandomCrop(( 360,640)),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensor()])
    composed_transforms_tr3 = transforms.Compose([
        RandomHorizontalFlip(),
        RandomScale((0.2, .8)),
        RandomCrop(( 720,1280)),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensor()])
    
    composed_transforms_ts = transforms.Compose([
        RandomHorizontalFlip(),
        #augment.Scale((819, 1638)),
        CenterCrop(( 512,1024)),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),#augment. Normalize_cityscapes(mean=(72.39, 82.91, 73.16)),
        ToTensor()])
#augment train, val data load
#cityscapes. 삭제
    cityscapes_train = Cityscapes(root=dataset_path,extra=CONFIG.USING_COARSE,split='train',transform=composed_transforms_tr)
    cityscapes_train1 = Cityscapes(root=dataset_path,extra=CONFIG.USING_COARSE,split='train',transform=composed_transforms_tr1)
    cityscapes_train2 = Cityscapes(root=dataset_path,extra=CONFIG.USING_COARSE,split='train',transform=composed_transforms_tr2)
    cityscapes_train3 = Cityscapes(root=dataset_path,extra=CONFIG.USING_COARSE,split='train',transform=composed_transforms_tr3)
    
    cityscapes_val = Cityscapes(root=dataset_path,extra=CONFIG.USING_COARSE,split='val', transform=composed_transforms_ts)
    
#augment train, val data load , train batch ==4
    trainloader = DataLoader(cityscapes_train, batch_size=p['trainBatch'], shuffle=True, num_workers=0)
    trainloader1 = DataLoader(cityscapes_train1, batch_size=p['trainBatch'], shuffle=True, num_workers=0)
    trainloader2 = DataLoader(cityscapes_train2, batch_size=p['trainBatch'], shuffle=True, num_workers=0)
    trainloader3 = DataLoader(cityscapes_train3, batch_size=p['trainBatch'], shuffle=True, num_workers=0)


    valloader = DataLoader(cityscapes_val, batch_size=testBatch, shuffle=True, num_workers=0)
    
    if CONFIG.USING_COARSE:#in case of training coarse data, I just used one scale to train. 상황에 따라 일반적인 tarin data를 쓸지, augment한 train data를 쓸지 정함 
        loaders=[ trainloader ]
    else:
        loaders=[ trainloader ,trainloader1 ,trainloader2 ,trainloader3]
    #generate 앞에 utils. 삭제
    generate_param_report(os.path.join(save_path, exp_name + '.txt'), p)

    num_img_tr = len(trainloader)
    num_img_vl = len(valloader)
    running_loss_tr = 0.0
    running_loss_vl = 0.0
    previous_miou = -1.0
    global_step = 0
    iev = Eval(20,19)
    
    # Main Training and Testing Loop 메인 학습 0부터 100까지
    for epoch in range(resume_epoch, nEpochs):
        start_time = timeit.default_timer()

        if epoch % p['epoch_size'] == p['epoch_size'] - 1:
            lr_ = lr_poly(p['lr'], epoch, nEpochs, 0.9)
            print('(poly lr policy) learning rate: ', lr_)
            optimizer = optim.SGD(net.parameters(), lr=lr_, momentum=p['momentum'], weight_decay=p['wd'])
           
        net.train()
        for loader in loaders:
            print(loader)
            for ii, sample_batched in enumerate(loader):
                
    
                inputs, labels = sample_batched['image'], sample_batched['label']
                # Forward-Backward of the mini-batch
                inputs, labels = Variable(inputs, requires_grad=True), Variable(labels)
                #print('labels size', inputs.size() , labels.size())
                global_step += inputs.data.shape[0]
                #print("Glopal Step",global_step)4,8,12,16
                if CONFIG.USING_GPU:
                    inputs, labels = inputs.cuda(), labels.cuda()
                
                optimizer.zero_grad()
                outputs = net.forward(inputs)
                loss = criterion(outputs, labels,reduct='sum',weight=None)#sum
                loss.backward()
                optimizer.step()
                ls=loss.item()
                running_loss_tr += ls
    #            if ii% 10 == 0:
    #                print(ls)
                # Print stuff
                if ii % num_img_tr == (num_img_tr - 1):
                    running_loss_tr = running_loss_tr / num_img_tr
                    writer.add_scalar('data/total_loss_epoch', running_loss_tr, epoch)
                    print('[Epoch: %d, numImages: %5d]' % (epoch, ii * p['trainBatch'] + inputs.data.shape[0]))
                    print('Loss: %f' % running_loss_tr)
                    running_loss_tr = 0
                    stop_time = timeit.default_timer()
                    print("Execution time: " + str(stop_time - start_time) + "\n")
       
                
                # Update the weights once in p['nAveGrad'] forward passes 
                writer.add_scalar('data/total_loss_iter', loss.item(), ii + num_img_tr * epoch)
                
    
                # Show 10 * 3 images results each epoch
                if ii % (num_img_tr // 10) == 0:
                    grid_image = make_grid(inputs[:3].clone().cpu().data, 3, normalize=True)
                    writer.add_image('Image', grid_image, global_step)
                    grid_image = make_grid(
                        decode_seg_map_sequence(torch.max(outputs[:3], 1)[1].detach().cpu().numpy(), 'cityscapes'), 3,
                        normalize=False,
                        range=(0, 255))
                    writer.add_image('Predicted label', grid_image, global_step)
                    grid_image = make_grid(
                        decode_seg_map_sequence(torch.squeeze(labels[:3], 1).detach().cpu().numpy(), 'cityscapes'), 3,
                        normalize=False, range=(0, 255))
                    writer.add_image('Groundtruth label', grid_image, global_step)

        # One testing epoch
        if (epoch % nValInterval == (nValInterval - 1)) or epoch==0:
            total_miou = 0.0
            net.eval()
            for ii, sample_batched in enumerate(valloader):
                inputs, labels = sample_batched['image'], sample_batched['label']
                print(inputs.shape)
                # Forward pass of the mini-batch
                inputs, labels = Variable(inputs, requires_grad=True), Variable(labels)
                if CONFIG.USING_GPU:
                    inputs, labels = inputs.cuda(), labels.cuda()

                with torch.no_grad():
                    outputs = net.forward(inputs)

                predictions = torch.max(outputs, 1)[1]

                loss = criterion(outputs, labels,reduct='sum',weight=None)#sum elementwise_mean
                running_loss_vl += loss.item()
               
                
                y = torch.ones(labels.size()[2], labels.size()[3]).mul(19).cuda()
                labels=labels.where(labels !=255, y)
                
                iev.addBatch(predictions.unsqueeze(1).data,labels)
                
                
                # Print stuff
                if ii % num_img_vl == num_img_vl - 1:
                    miou=iev.getIoU()[0]
                    running_loss_vl = running_loss_vl / num_img_vl
                    print('Validation:')
                    print('[Epoch: %d, numImages: %5d]' % (epoch, ii * testBatch + inputs.data.shape[0]))
                    writer.add_scalar('data/test_loss_epoch', running_loss_vl, epoch)
                    writer.add_scalar('data/test_miour', iev.getIoU()[0], epoch)
                    print('Loss: %f' % running_loss_vl)
                    print("Predi iou",iev.getIoU())
                    running_loss_vl = 0
                    iev.reset()

        # Save the model
        if (epoch % snapshot) == snapshot - 1 :#and miou > previous_miou
            previous_miou = miou
            torch.save(net.state_dict(), os.path.join(save_path, 'models', modelName + '_epoch-' + str(epoch) + '.pth'))
            print("Save model at {}\n".format(
                os.path.join(save_path, 'models', modelName + '_epoch-' + str(epoch) + '.pth')))

    writer.close()


'\nimport socket\nimport timeit\nfrom datetime import datetime\nimport os\nimport glob\nfrom collections import OrderedDict\nimport numpy as np\nimport yaml\nfrom addict import Dict\nimport argparse\n\n# PyTorch includes\nimport torch\nfrom torch.autograd import Variable\nimport torch.optim as optim\nfrom torchvision import transforms\nfrom torch.utils.data import DataLoader\nfrom torchvision.utils import make_grid\n\n# Tensorboard include\nfrom tensorboardX import SummaryWriter\n\n# Custom includes\n#from dataloaders import cityscapes\n#from dataloaders import utils\n#from dataloaders import augmentation as augment\n#from models.liteseg import LiteSeg\n#from utils import loss as losses\n#from utils import iou_eval\n\n\n#To make reproducible results  \ntorch.manual_seed(125)\ntorch.backends.cudnn.deterministic = True\ntorch.backends.cudnn.benchmark = False\nnp.random.seed(125)\n\n#경로 변경 주의\nCONFIG=Dict(yaml.load(open(\'/home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/config/training.ya

## mobilenetv2
- pretrained download : https://github.com/d-li14/mobilenetv2.pytorch/blob/master/pretrained/mobilenetv2_1.0-0c6065bc.pth
- 경로 설정 주의

In [21]:
import socket
import timeit
from datetime import datetime
import os
import glob
from collections import OrderedDict
import numpy as np
import yaml
from addict import Dict
import argparse

# PyTorch includes
import torch
from torch.autograd import Variable
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

# Tensorboard include
from tensorboardX import SummaryWriter

# Custom includes
#from dataloaders import cityscapes
#from dataloaders import utils
#from dataloaders import augmentation as augment
#from models.liteseg import LiteSeg
#from utils import loss as losses
#from utils import iou_eval


#To make reproducible results  
torch.manual_seed(125)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(125)

#경로 변경 주의
CONFIG=Dict(yaml.load(open('/home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/config/training.yaml')))

#ap, args 함수 : shell 창에서 어떤 backbone 을 쓸지, model path, pretrained_mode
#ap = argparse.ArgumentParser()
#ap.add_argument('--backbone_network', required=True,
                #help = 'name of backbone network',default='darknet')#shufflenet, mobilenet, and darknet
#ap.add_argument('--model_path_coarse', required=False,
                #help = 'path to pretrained model on coarse data',default='pretrained_models/liteseg-darknet-cityscapes.pth')
#ap.add_argument('--model_path_resume', required=False,
                #help = 'path to a model to resume from',default='pretrained_models/liteseg-darknet-cityscapes.pth')

#args = ap.parse_args()
backbone_network= 'mobilenet'
#backbone_network= 'darknet' # backbone_network 뭘로 할지 str로 입력
#model_path_resume=args.model_path_resume
#경로 변경 주의
#model_path_coarse ='/home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/pretrained_models/liteseg-darknet-cityscapes.pth'
model_path_coarse = '/home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/pretrained_models/liteseg-mobilenet-cityscapes.pth'
#model_path_resume = '/home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/pretrained_models/liteseg-darknet-cityscapes.pth'


# Setting parameters
nEpochs =10  # Number of epochs for training 150 종료epoch
resume_epoch = 0  # Default is 0, change if want to resume 0 시작epoch

p = OrderedDict()  # Parameters to include in report
p['trainBatch'] =4  # Training batch size
p['lr'] =1e-7 # Learning rate  1e-8 for darknet and 1e-7 shufflenet and mobilenet
p['wd'] = 5e-4  # Weight decay
p['momentum'] = 0.9  # Momentum
p['epoch_size'] =5  # epochs to change learning rate

testBatch = 1  # Testing batch size
nValInterval = 2  # Run on test set every nTestInterval epochs
snapshot = 2  # Store a model every snapshot epochs

# 저장 경로 설정
save_dir_root = os.getenv('HOME')+'/aiffel/siaiffel/LiteSeg-master/handmade_weights'

dataset_path=CONFIG.DATASET_FINE
if CONFIG.USING_COARSE:
    print("Training on Coarse Data")
    dataset_path=CONFIG.DATASET_COARSE
    p['epoch_size'] =10 #we increase the number of epochs to change LR as we train on one scale


#txt파일 이름 설정
# exp_name = os.path.dirname(os.path.abspath(__file__)).split('/')[-1]
exp_name = 'sangmin_0520'

class_weight = np.array([0.05570516, 0.32337477, 0.08998544, 1.03602707, 1.03413147, 1.68195437,
                                 5.58540548, 3.56563995, 0.12704978, 1.,         0.46783719, 1.34551528,
                                 5.29974114, 0.28342531, 0.9396095,  0.81551811, 0.42679146, 3.6399074,
                                 2.78376194], dtype=float)
class_weight = torch.from_numpy(class_weight).float().cuda()



#make a folder -with name of current time- for every experiment
experiment_id=datetime.now().strftime("%Y-%m-%d_%H_%M")
save_path = os.path.join(save_dir_root, 'experiments', 'experiment_' + str(experiment_id))
print(save_path)


# Network definition
# LiteSeg.build 괄호 첫번째에 backbone_network 뭘로 할지 str로 입력
modelpath = model_path_coarse
net=LiteSeg.build(backbone_network, modelpath ,CONFIG, is_train=True)

if CONFIG.USING_GPU:
    torch.cuda.set_device(device=CONFIG.GPU_ID)
    net.cuda()


#using the trained model on the coarse data
#If you want to train model on fine data directley, comment the next 3 lines.
if not CONFIG.USING_COARSE:
    print("Using a weights from training coarse data from: {}...".format(model_path_coarse))
    net.load_state_dict(torch.load(model_path_coarse)) 


#resume tarining from a given model, 
#Attention! the learnig rate which used for resuming training, is not the intial one.
if resume_epoch == 0:
    print("Training Network...")
else:
    print("Resume training from a model at: {}...".format(model_path_resume))
    net.load_state_dict(torch.load(model_path_resume))

    
modelName = 'LiteSeg-' + backbone_network + '-cityscapes'
print(modelName)

criterion = cross_entropy2d

# 여기서부터 시작
if resume_epoch != nEpochs+1:
    # Logging into Tensorboard / path 설정
    log_dir = os.path.join(save_path, 'models', datetime.now().strftime('%b%d_%H-%M-%S') + '_' + socket.gethostname())
    writer = SummaryWriter(log_dir=log_dir) #summary 설정

    # Use the following optimizer 옵티마이저 설정
    optimizer = optim.SGD(net.parameters(), lr=p['lr'], momentum=p['momentum'], weight_decay=p['wd'])
    #optimizer = optim.Adam(net.parameters(), 5e-4, (0.9, 0.999), eps=1e-08, weight_decay=1e-4) 
    p['optimizer'] = str(optimizer) 

    #augment
    #augment. 삭제
    composed_transforms_tr = transforms.Compose([
        RandomHorizontalFlip(),
        RandomScale((0.2, .8)),
        RandomCrop(( 512,1024)),
        RandomRotate(5),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensor()])
   
    composed_transforms_tr1 = transforms.Compose([
        RandomHorizontalFlip(),
        RandomScale((0.2, .8)),
        RandomCrop(( 768,1536)),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensor()])
    composed_transforms_tr2 = transforms.Compose([
        RandomHorizontalFlip(),
        RandomScale((0.2, .8)),
        RandomCrop(( 360,640)),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensor()])
    composed_transforms_tr3 = transforms.Compose([
        RandomHorizontalFlip(),
        RandomScale((0.2, .8)),
        RandomCrop(( 720,1280)),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensor()])
    
    composed_transforms_ts = transforms.Compose([
        RandomHorizontalFlip(),
        #augment.Scale((819, 1638)),
        CenterCrop(( 512,1024)),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),#augment. Normalize_cityscapes(mean=(72.39, 82.91, 73.16)),
        ToTensor()])
#augment train, val data load
#cityscapes. 삭제
    cityscapes_train = Cityscapes(root=dataset_path,extra=CONFIG.USING_COARSE,split='train',transform=composed_transforms_tr)
    cityscapes_train1 = Cityscapes(root=dataset_path,extra=CONFIG.USING_COARSE,split='train',transform=composed_transforms_tr1)
    cityscapes_train2 = Cityscapes(root=dataset_path,extra=CONFIG.USING_COARSE,split='train',transform=composed_transforms_tr2)
    cityscapes_train3 = Cityscapes(root=dataset_path,extra=CONFIG.USING_COARSE,split='train',transform=composed_transforms_tr3)
    
    cityscapes_val = Cityscapes(root=dataset_path,extra=CONFIG.USING_COARSE,split='val', transform=composed_transforms_ts)
    
#augment train, val data load , train batch ==4
    trainloader = DataLoader(cityscapes_train, batch_size=p['trainBatch'], shuffle=True, num_workers=0)
    trainloader1 = DataLoader(cityscapes_train1, batch_size=p['trainBatch'], shuffle=True, num_workers=0)
    trainloader2 = DataLoader(cityscapes_train2, batch_size=p['trainBatch'], shuffle=True, num_workers=0)
    trainloader3 = DataLoader(cityscapes_train3, batch_size=p['trainBatch'], shuffle=True, num_workers=0)


    valloader = DataLoader(cityscapes_val, batch_size=testBatch, shuffle=True, num_workers=0)
    
    if CONFIG.USING_COARSE:#in case of training coarse data, I just used one scale to train. 상황에 따라 일반적인 tarin data를 쓸지, augment한 train data를 쓸지 정함 
        loaders=[ trainloader ]
    else:
        loaders=[ trainloader ,trainloader1 ,trainloader2 ,trainloader3]
    #generate 앞에 utils. 삭제
    generate_param_report(os.path.join(save_path, exp_name + '.txt'), p)

    num_img_tr = len(trainloader)
    num_img_vl = len(valloader)
    running_loss_tr = 0.0
    running_loss_vl = 0.0
    previous_miou = -1.0
    global_step = 0
    iev = Eval(20,19)
    
    # Main Training and Testing Loop 메인 학습 0부터 100까지
    for epoch in range(resume_epoch, nEpochs):
        start_time = timeit.default_timer()

        if epoch % p['epoch_size'] == p['epoch_size'] - 1:
            lr_ = lr_poly(p['lr'], epoch, nEpochs, 0.9)
            print('(poly lr policy) learning rate: ', lr_)
            optimizer = optim.SGD(net.parameters(), lr=lr_, momentum=p['momentum'], weight_decay=p['wd'])
           
        net.train()
        for loader in loaders:
            print(loader)
            for ii, sample_batched in enumerate(loader):
                
    
                inputs, labels = sample_batched['image'], sample_batched['label']
                # Forward-Backward of the mini-batch
                inputs, labels = Variable(inputs, requires_grad=True), Variable(labels)
                #print('labels size', inputs.size() , labels.size())
                global_step += inputs.data.shape[0]
                #print("Glopal Step",global_step)4,8,12,16
                if CONFIG.USING_GPU:
                    inputs, labels = inputs.cuda(), labels.cuda()
                
                optimizer.zero_grad()
                outputs = net.forward(inputs)
                loss = criterion(outputs, labels,reduct='sum',weight=None)#sum
                loss.backward()
                optimizer.step()
                ls=loss.item()
                running_loss_tr += ls
    #            if ii% 10 == 0:
    #                print(ls)
                # Print stuff
                if ii % num_img_tr == (num_img_tr - 1):
                    running_loss_tr = running_loss_tr / num_img_tr
                    writer.add_scalar('data/total_loss_epoch', running_loss_tr, epoch)
                    print('[Epoch: %d, numImages: %5d]' % (epoch, ii * p['trainBatch'] + inputs.data.shape[0]))
                    print('Loss: %f' % running_loss_tr)
                    running_loss_tr = 0
                    stop_time = timeit.default_timer()
                    print("Execution time: " + str(stop_time - start_time) + "\n")
       
                
                # Update the weights once in p['nAveGrad'] forward passes 
                writer.add_scalar('data/total_loss_iter', loss.item(), ii + num_img_tr * epoch)
                
    
                # Show 10 * 3 images results each epoch
                if ii % (num_img_tr // 10) == 0:
                    grid_image = make_grid(inputs[:3].clone().cpu().data, 3, normalize=True)
                    writer.add_image('Image', grid_image, global_step)
                    grid_image = make_grid(
                        decode_seg_map_sequence(torch.max(outputs[:3], 1)[1].detach().cpu().numpy(), 'cityscapes'), 3,
                        normalize=False,
                        range=(0, 255))
                    writer.add_image('Predicted label', grid_image, global_step)
                    grid_image = make_grid(
                        decode_seg_map_sequence(torch.squeeze(labels[:3], 1).detach().cpu().numpy(), 'cityscapes'), 3,
                        normalize=False, range=(0, 255))
                    writer.add_image('Groundtruth label', grid_image, global_step)

        # One testing epoch
        if (epoch % nValInterval == (nValInterval - 1)) or epoch==0:
            total_miou = 0.0
            net.eval()
            for ii, sample_batched in enumerate(valloader):
                inputs, labels = sample_batched['image'], sample_batched['label']

                # Forward pass of the mini-batch
                inputs, labels = Variable(inputs, requires_grad=True), Variable(labels)
                if CONFIG.USING_GPU:
                    inputs, labels = inputs.cuda(), labels.cuda()

                with torch.no_grad():
                    outputs = net.forward(inputs)

                predictions = torch.max(outputs, 1)[1]

                loss = criterion(outputs, labels,reduct='sum',weight=None)#sum elementwise_mean
                running_loss_vl += loss.item()
               
                
                y = torch.ones(labels.size()[2], labels.size()[3]).mul(19).cuda()
                labels=labels.where(labels !=255, y)
                
                iev.addBatch(predictions.unsqueeze(1).data,labels)
                
                
                # Print stuff
                if ii % num_img_vl == num_img_vl - 1:
                    miou=iev.getIoU()[0]
                    running_loss_vl = running_loss_vl / num_img_vl
                    print('Validation:')
                    print('[Epoch: %d, numImages: %5d]' % (epoch, ii * testBatch + inputs.data.shape[0]))
                    writer.add_scalar('data/test_loss_epoch', running_loss_vl, epoch)
                    writer.add_scalar('data/test_miour', iev.getIoU()[0], epoch)
                    print('Loss: %f' % running_loss_vl)
                    print("Predi iou",iev.getIoU())
                    running_loss_vl = 0
                    iev.reset()

        # Save the model
        if (epoch % snapshot) == snapshot - 1 :#and miou > previous_miou
            previous_miou = miou
            torch.save(net.state_dict(), os.path.join(save_path, 'models', modelName + '_epoch-' + str(epoch) + '.pth'))
            print("Save model at {}\n".format(
                os.path.join(save_path, 'models', modelName + '_epoch-' + str(epoch) + '.pth')))

    writer.close()


/home/aiffel-dj17/anaconda3/envs/sia/lib/python3.7/site-packages/ipykernel_launcher.py:39: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


/home/aiffel-dj17/aiffel/siaiffel/LiteSeg-master/handmade_weights/experiments/experiment_2021-05-21_16_24
LiteSeg-MobileNet...


RuntimeError: Error(s) in loading state_dict for MobileNetV2:
	Missing key(s) in state_dict: "features.0.0.weight", "features.0.1.weight", "features.0.1.bias", "features.0.1.running_mean", "features.0.1.running_var", "features.1.conv.0.weight", "features.1.conv.1.weight", "features.1.conv.1.bias", "features.1.conv.1.running_mean", "features.1.conv.1.running_var", "features.1.conv.3.weight", "features.1.conv.4.weight", "features.1.conv.4.bias", "features.1.conv.4.running_mean", "features.1.conv.4.running_var", "features.2.conv.0.weight", "features.2.conv.1.weight", "features.2.conv.1.bias", "features.2.conv.1.running_mean", "features.2.conv.1.running_var", "features.2.conv.3.weight", "features.2.conv.4.weight", "features.2.conv.4.bias", "features.2.conv.4.running_mean", "features.2.conv.4.running_var", "features.2.conv.6.weight", "features.2.conv.7.weight", "features.2.conv.7.bias", "features.2.conv.7.running_mean", "features.2.conv.7.running_var", "features.3.conv.0.weight", "features.

In [ ]:
import brambox